In [1]:
!pip install gensim
!pip install python-levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 6.5 MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149873 sha256=45023428d9210f311c633fd9128b1d46ab601bb4a78e64ac05fe6043ed6326bb
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-levenshtein


The dataset we are using here is a subset of Amazon reviews from the Cell Phones & Accessories category. The data is stored as a JSON file and can be read using pandas.

Link to the Dataset:
[Review Cell Phones On Amazon](http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz)

In [2]:
import gensim
import pandas as pd


In [13]:
df = pd.read_json("/content/reviews_Cell_Phones_and_Accessories_5.json",lines=True)

In [14]:
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [15]:
df.shape

(194439, 9)

In [17]:
df['reviewText']

0         They look good and stick good! I just don't li...
1         These stickers work like the review says they ...
2         These are awesome and make my phone look so st...
3         Item arrived in great time and was in perfect ...
4         awesome! stays on, and looks great. can be use...
                                ...                        
194434    Works great just like my original one. I reall...
194435    Great product. Great packaging. High quality a...
194436    This is a great cable, just as good as the mor...
194437    I really like it becasue it works well with my...
194438    product as described, I have wasted a lot of m...
Name: reviewText, Length: 194439, dtype: object

word2vec training by only review text.

In [18]:
#preprocess the review text using gensim

review_text = df.reviewText.apply(gensim.utils.simple_preprocess) #It will create a list for each review of tokenize words
review_text



0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [19]:
model = gensim.models.Word2Vec(window = 10,min_count=2,workers=4) #window =when i say 10 it means,10 word before target word ,10 word after target word ,min_count=2 atlest 2 word should be present inorder to consider for training,workers = how many cpu trade you want

In [20]:
#Build vocab

model.build_vocab(review_text)

In [21]:
model.train(review_text,total_examples=model.corpus_count,epochs=model.epochs)

(61503919, 83868975)

In [22]:
model.save("./word2vec-amazon-cell-accessories-reviews")

In [23]:
model.wv.most_similar("bad")

[('terrible', 0.6663188338279724),
 ('shabby', 0.6457138657569885),
 ('horrible', 0.6230558753013611),
 ('good', 0.5790437459945679),
 ('awful', 0.5623327493667603),
 ('okay', 0.5527682900428772),
 ('crappy', 0.5503761172294617),
 ('disappointing', 0.5412116646766663),
 ('sad', 0.5394091010093689),
 ('funny', 0.5310283303260803)]

In [25]:
print(model.predict_output_word(['they','look','and','stick']))

[('forward', 0.0014936848), ('professional', 0.0013088902), ('classy', 0.0009080286), ('closely', 0.00088034105), ('elsewhere', 0.00068684714), ('sharp', 0.00067717955), ('won', 0.0005629708), ('tacky', 0.0005334113), ('clean', 0.0004872882), ('are', 0.00048708983)]
